In [ ]:
!pip install google-generativeai gradio plotly pandas cryptography


In [ ]:
pip install gradio_client

In [ ]:
import gradio as gr
import json
import datetime
import pandas as pd
import plotly.express as px
import sqlite3
from typing import Dict, List
from dataclasses import dataclass
from cryptography.fernet import Fernet
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# -------------------------
# User Profile Data Class
# -------------------------
@dataclass
class UserProfile:
    name: str = ""
    age: int = 0
    occupation: str = ""
    income: float = 0
    risk_tolerance: str = "moderate"  # conservative, moderate, aggressive
    financial_goals: List[str] = None
    language: str = "en"
    communication_style: str = "professional"  # student, professional, casual

# -------------------------
# Finance Data Manager
# -------------------------
class FinanceDataManager:
    def __init__(self):
        self.db_path = "finance_data.db"
        self.encryption_key = Fernet.generate_key()
        self.cipher_suite = Fernet(self.encryption_key)
        self.init_database()

    def init_database(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_profiles (
                id INTEGER PRIMARY KEY,
                encrypted_data TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS transactions (
                id INTEGER PRIMARY KEY,
                user_id INTEGER,
                encrypted_data TEXT,
                date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS goals (
                id INTEGER PRIMARY KEY,
                user_id INTEGER,
                encrypted_data TEXT,
                created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        conn.commit()
        conn.close()

    def encrypt_data(self, data: str) -> str:
        return self.cipher_suite.encrypt(data.encode()).decode()

    def decrypt_data(self, encrypted_data: str) -> str:
        return self.cipher_suite.decrypt(encrypted_data.encode()).decode()

    # --- Goal Management ---
    def save_goal(self, user_id: int, goal: str):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        encrypted_goal = self.encrypt_data(goal)
        cursor.execute("INSERT INTO goals (user_id, encrypted_data) VALUES (?, ?)", (user_id, encrypted_goal))
        conn.commit()
        conn.close()

    def get_goals(self, user_id: int):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT encrypted_data, created_date FROM goals WHERE user_id=?", (user_id,))
        rows = cursor.fetchall()
        conn.close()
        return [(self.decrypt_data(row[0]), row[1]) for row in rows]

# -------------------------
# Chatbot with IBM Granite
# -------------------------
class PersonalFinanceChatbot:
    def __init__(self):
        self.user_profile = UserProfile()
        self.data_manager = FinanceDataManager()
        self.conversation_history = []
        self.granite_model = None
        self.granite_tokenizer = None

    def initialize_granite(self):
        """Load IBM Granite model"""
        model_path = "ibm-granite/granite-3.3-2b-instruct"
        self.granite_tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.granite_model = AutoModelForCausalLM.from_pretrained(
            model_path, device_map="auto", torch_dtype="auto"
        )
        self.granite_model.eval()

    def get_communication_style_prompt(self) -> str:
        if self.user_profile.communication_style == "student":
            return "Use simple, educational language with examples. Be encouraging and patient."
        elif self.user_profile.communication_style == "professional":
            return "Use professional, concise language with technical terms when appropriate."
        else:
            return "Use friendly, conversational language that's easy to understand."

    def create_context_prompt(self, user_input: str) -> str:
        return f"""
        You are a personal finance advisor chatbot. Here's the user profile:
        - Age: {self.user_profile.age}
        - Occupation: {self.user_profile.occupation}
        - Income: ${self.user_profile.income:,.2f}
        - Risk Tolerance: {self.user_profile.risk_tolerance}
        - Communication Style: {self.get_communication_style_prompt()}

        Please provide personalized financial advice based on this profile.
        Focus on: budgeting, savings, investments, tax planning, and goal setting.

        User Question: {user_input}
        """

    def process_natural_language_query(self, query: str) -> str:
        """Process query using IBM Granite"""
        if not self.granite_model:
            self.initialize_granite()

        conv = [
            {"role": "system", "content": self.create_context_prompt(query)},
            {"role": "user", "content": query}
        ]
        inputs = self.granite_tokenizer.apply_chat_template(
            conv,
            return_tensors="pt",
            thinking=True,
            return_dict=True,
            add_generation_prompt=True
        ).to(self.granite_model.device)

        set_seed(42)
        output = self.granite_model.generate(
            **inputs, max_new_tokens=512, temperature=0.7
        )
        reply = self.granite_tokenizer.decode(
            output[0, inputs["input_ids"].shape[1]:], skip_special_tokens=True
        )

        self.conversation_history.append({
            "user": query,
            "bot": reply,
            "timestamp": datetime.datetime.now().isoformat()
        })
        return reply

    # --- Budget Analyzer ---
    def analyze_budget(self, income: float, expenses: Dict[str, float]) -> Dict:
        total_expenses = sum(expenses.values())
        savings = income - total_expenses
        savings_rate = (savings / income) * 100 if income > 0 else 0
        return {
            "total_income": income,
            "total_expenses": total_expenses,
            "savings": savings,
            "savings_rate": savings_rate
        }

    def create_budget_visualization(self, expenses: Dict[str, float]):
        if not expenses:
            return None
        fig = px.pie(
            values=list(expenses.values()),
            names=list(expenses.keys()),
            title="Expense Breakdown"
        )
        return fig

    # --- Investment Recommendations ---
    def get_investment_recommendations(self):
        if self.user_profile.risk_tolerance == "conservative":
            return {
                "Bonds / Fixed Income": "60%",
                "Index Funds / ETFs": "30%",
                "Stocks": "10%"
            }
        elif self.user_profile.risk_tolerance == "moderate":
            return {
                "Bonds / Fixed Income": "40%",
                "Index Funds / ETFs": "40%",
                "Stocks": "20%"
            }
        else:  # aggressive
            return {
                "Bonds / Fixed Income": "20%",
                "Index Funds / ETFs": "30%",
                "Stocks": "50%"
            }

# -------------------------
# Gradio Interface
# -------------------------
def create_gradio_interface():
    chatbot = PersonalFinanceChatbot()

    def chat_interface(message, history):
        response = chatbot.process_natural_language_query(message)
        history.append((message, response))
        return history, ""

    def setup_profile(name, age, occupation, income, risk_tolerance, comm_style):
        chatbot.user_profile.name = name
        chatbot.user_profile.age = int(age) if age else 0
        chatbot.user_profile.occupation = occupation
        chatbot.user_profile.income = float(income) if income else 0
        chatbot.user_profile.risk_tolerance = risk_tolerance
        chatbot.user_profile.communication_style = comm_style
        return f"✅ Profile updated for {name}!"

    def analyze_budget_interface(income, housing, food, transport, utilities, entertainment, other):
        expenses = {
            "Housing": float(housing) if housing else 0,
            "Food": float(food) if food else 0,
            "Transportation": float(transport) if transport else 0,
            "Utilities": float(utilities) if utilities else 0,
            "Entertainment": float(entertainment) if entertainment else 0,
            "Other": float(other) if other else 0
        }
        analysis = chatbot.analyze_budget(float(income), expenses)
        chart = chatbot.create_budget_visualization(expenses)
        summary = f"""
📊 BUDGET ANALYSIS

💰 Income: ${analysis['total_income']:,.2f}
💸 Expenses: ${analysis['total_expenses']:,.2f}
💵 Savings: ${analysis['savings']:,.2f}
📈 Savings Rate: {analysis['savings_rate']:.1f}%
"""
        return summary, chart

    def add_goal_interface(goal):
        chatbot.data_manager.save_goal(user_id=1, goal=goal)  # single-user for now
        goals = chatbot.data_manager.get_goals(user_id=1)
        return goals

    def investment_interface():
        recs = chatbot.get_investment_recommendations()
        summary = "📊 Recommended Portfolio:\n\n"
        for k, v in recs.items():
            summary += f"- {k}: {v}\n"
        return summary

    with gr.Blocks(theme=gr.themes.Soft(), title="Personal Finance AI Assistant") as demo:
        gr.Markdown("# 🏦 Personal Finance AI Assistant\n### Powered by IBM Granite AI")

        with gr.Tab("💬 AI Chat"):
            chatbot_interface = gr.Chatbot(height=400, label="Financial AI Assistant")
            msg = gr.Textbox(label="Ask me anything about personal finance...")
            msg.submit(chat_interface, [msg, chatbot_interface], [chatbot_interface, msg])

        with gr.Tab("👤 Profile Setup"):
            with gr.Row():
                with gr.Column():
                    name_input = gr.Textbox(label="Name")
                    age_input = gr.Number(label="Age", value=25)
                    occupation_input = gr.Textbox(label="Occupation")
                with gr.Column():
                    income_input = gr.Number(label="Annual Income ($)", value=50000)
                    risk_input = gr.Dropdown(
                        choices=["conservative", "moderate", "aggressive"],
                        label="Risk Tolerance",
                        value="moderate"
                    )
                    style_input = gr.Dropdown(
                        choices=["student", "professional", "casual"],
                        label="Communication Style",
                        value="professional"
                    )
            setup_btn = gr.Button("💾 Save Profile")
            profile_status = gr.Textbox(label="Status", interactive=False)
            setup_btn.click(
                setup_profile,
                inputs=[name_input, age_input, occupation_input, income_input, risk_input, style_input],
                outputs=[profile_status]
            )

        with gr.Tab("📊 Budget Analyzer"):
            with gr.Row():
                with gr.Column():
                    budget_income = gr.Number(label="Monthly Income ($)", value=5000)
                    housing_exp = gr.Number(label="Housing ($)", value=1500)
                    food_exp = gr.Number(label="Food ($)", value=600)
                    transport_exp = gr.Number(label="Transportation ($)", value=400)
                with gr.Column():
                    utilities_exp = gr.Number(label="Utilities ($)", value=200)
                    entertainment_exp = gr.Number(label="Entertainment ($)", value=300)
                    other_exp = gr.Number(label="Other ($)", value=200)

            analyze_btn = gr.Button("🔍 Analyze Budget")
            budget_summary = gr.Textbox(label="Budget Summary", lines=8)
            budget_chart = gr.Plot()

            analyze_btn.click(
                analyze_budget_interface,
                inputs=[budget_income, housing_exp, food_exp, transport_exp, utilities_exp, entertainment_exp, other_exp],
                outputs=[budget_summary, budget_chart]
            )

        with gr.Tab("🎯 Goals"):
            goal_input = gr.Textbox(label="Enter your financial goal")
            add_goal_btn = gr.Button("➕ Add Goal")
            goals_list = gr.Dataframe(headers=["Goal", "Created Date"], interactive=False)
            add_goal_btn.click(add_goal_interface, inputs=[goal_input], outputs=[goals_list])

        with gr.Tab("📈 Investments"):
            rec_btn = gr.Button("💡 Show Recommendations")
            rec_output = gr.Textbox(label="Suggested Portfolio", lines=6)
            rec_btn.click(investment_interface, outputs=[rec_output])

    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch()

/tmp/ipython-input-4243806778.py:264: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(height=400, label="Financial AI Assistant")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d58c595330529b8c47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
